In [1]:
import numpy as np
import ROOT as rt
from tqdm import tqdm

### In this notebook, I am exploring the dependance of pT Hard on charm production

In [2]:
# Define excluded PDG codes
excluded_pdgs = {
    # Quarks
    1, 2, 3, 4, 5, 6,  # up through top
    -1, -2, -3, -4, -5, -6,  # anti-quarks
    
    # Gluons
    21,
    
    # Special codes
    90, 91, 92, 93,  # Cluster, string endpoints, etc
    
    # Diquarks
    1103, 2101, 2103, 2203, 2101, 2103, 2203, 3101, 3103, 3201, 3203,
    -1103, -2101, -2103, -2203, -2101, -2103, -2203, -3101, -3103, -3201, -3203
}

# Define charm hadron PDG codes without D* to account for D* decays
final_charm_hadrons = {
    411, -411,    # D+, D-
    421, -421,    # D0, D0-bar  
    431, -431,    # Ds+, Ds-
    4122, -4122,  # Λc+, Λc-
}
#     4132, -4132,  # Ξc0, Ξc0-bar
#     4232, -4232,  # Ξc+, Ξc-
#     4332, -4332,  # Ωc0, Ωc0-bar
#     # J/psi
#     443,
#     # Other charm mesons
#     423, -423,    # D*(2007)0, D*(2007)0-bar
#     413, -413,    # D*(2010)+, D*(2010)-
#     433, -433,    # D*_s+, D*_s-
#     10411, -10411, # D*_0(2400)+, D*_0(2400)-
#     10421, -10421, # D*_0(2400)0, D*_0(2400)0-bar
#     10413, -10413, # D_1(2420)+, D_1(2420)-
#     10423, -10423, # D_1(2420)0, D_1(2420)0-bar
#     20413, -20413, # D_1(2430)+, D_1(2430)-
#     20423, -20423, # D_1(2430)0, D_1(2430)0-bar
# }

charm_hadrons = {
    411, -411,    # D+, D-
    421, -421,    # D0, D0-bar  
    431, -431,    # Ds+, Ds-
    4122, -4122,  # Λc+, Λc-
    4132, -4132,  # Ξc0, Ξc0-bar
    4232, -4232,  # Ξc+, Ξc-
    4332, -4332,  # Ωc0, Ωc0-bar
    # J/psi
    443,
    # Other charm mesons
    423, -423,    # D*(2007)0, D*(2007)0-bar
    413, -413,    # D*(2010)+, D*(2010)-
    433, -433,    # D*_s+, D*_s-
    10411, -10411, # D*_0(2400)+, D*_0(2400)-
    10421, -10421, # D*_0(2400)0, D*_0(2400)0-bar
    10413, -10413, # D_1(2420)+, D_1(2420)-
    10423, -10423, # D_1(2420)0, D_1(2420)0-bar
    20413, -20413, # D_1(2430)+, D_1(2430)-
    20423, -20423, # D_1(2430)0, D_1(2430)0-bar
}

- note that i exlucded all charmed hadrons except d0 and d+

In [4]:
pt_hat_values = [0.0, 4.0, 10.0, 14.0]  # This will give you 8.0, 10.0, 12.0, 14.0
# pt_hat_values = [0.0]  # This will give you 8.0, 10.0, 12.0, 14.0


# Create histogram for charm hadrons per event for each ptHat value
histograms = []
colors = [rt.kRed, rt.kBlue, rt.kGreen+2, rt.kMagenta, rt.kOrange+7, 
          rt.kCyan+1, rt.kYellow+2, rt.kViolet+1, rt.kTeal+1, rt.kSpring+1, rt.kGray+2]


# Process each ptHat value
for pt_hat, color in zip(pt_hat_values, colors):
    filename = f"genpt{pt_hat:.1f}.root"
    print(f"\nProcessing {filename}")
    
    # Create histogram for this ptHat value
    hist = rt.TH1F(f"hist_pt{pt_hat:.1f}", "Charm Hadrons per Event;Charm Hadrons per Event;Count", 20, 0, 20)
    hist.SetLineColor(color)
    hist.SetLineWidth(2)
    
    # Open ROOT file and get tree
    f = rt.TFile(filename)
    tree = f.Get("EventTree")
    
    # Create TClonesArray to hold particles
    particles = rt.TClonesArray("TParticle")
    tree.SetBranchAddress("Particles", particles)
    
    # Process events
    total_charm = 0
    total_events = tree.GetEntries()
    events_with_charm = 0
    
    for i in tqdm(range(total_events), desc=f"ptHat {pt_hat:.1f}"):
        tree.GetEntry(i)
        charm_in_event = 0
        
        # Count charm hadrons in this event
        for j in range(particles.GetEntries()):
            p = particles.At(j)
            if abs(p.GetPdgCode()) in final_charm_hadrons:
                charm_in_event += 1
        
        hist.Fill(charm_in_event)
        total_charm += charm_in_event
        if charm_in_event > 0:
            events_with_charm += 1
    
    histograms.append(hist)
    
    # Print statistics for this ptHat value
    print(f"ptHat = {pt_hat:.1f}:")
    print(f"  Total events: {total_events}")
    print(f"  Events with charm: {events_with_charm} ({events_with_charm/total_events*100:.2f}%)")
    print(f"  Total charm hadrons: {total_charm}")
    print(f"  Average charm hadrons per event: {total_charm/total_events:.4f}")
    
    f.Close()

# Create canvas and draw overlaid histograms
c = rt.TCanvas("c", "Charm Hadrons per Event - ptHard Comparison", 800, 600)
c.SetLogy()

# Create legend
legend = rt.TLegend(0.7, 0.7, 0.9, 0.9)
legend.SetHeader("ptHard values")

# Draw histograms
for i, (hist, pt_hat) in enumerate(zip(histograms, pt_hat_values)):
    if i == 0:
        hist.SetMaximum(1100)
        hist.Draw()
    else:
        hist.Draw("SAME")
    legend.AddEntry(hist, f"ptHard = {pt_hat:.1f}", "l")

legend.Draw()
c.Draw()


Processing genpt0.0.root


ptHat 0.0: 100%|██████████| 1000/1000 [00:00<00:00, 2325.50it/s]


ptHat = 0.0:
  Total events: 1000
  Events with charm: 33 (3.30%)
  Total charm hadrons: 69
  Average charm hadrons per event: 0.0690

Processing genpt4.0.root


ptHat 4.0: 100%|██████████| 1000/1000 [00:01<00:00, 706.07it/s]


ptHat = 4.0:
  Total events: 1000
  Events with charm: 203 (20.30%)
  Total charm hadrons: 491
  Average charm hadrons per event: 0.4910

Processing genpt10.0.root


ptHat 10.0: 100%|██████████| 1000/1000 [00:02<00:00, 428.31it/s]


ptHat = 10.0:
  Total events: 1000
  Events with charm: 394 (39.40%)
  Total charm hadrons: 1067
  Average charm hadrons per event: 1.0670

Processing genpt14.0.root


ptHat 14.0: 100%|██████████| 1000/1000 [00:02<00:00, 446.75it/s]


ptHat = 14.0:
  Total events: 1000
  Events with charm: 416 (41.60%)
  Total charm hadrons: 1131
  Average charm hadrons per event: 1.1310


Warning in <TROOT::Append>: Replacing existing TH1: hist_pt0.0 (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


lets analyze why is there 3 charmed hadrons existing?
seems to be flavor excitation

In [11]:
# pt_hat_values = [0.0, 2.0, 4.0, 8.0, 14.0]  # This will give you 8.0, 10.0, 12.0, 14.0
pt_hat_values = [4.0]  # This will give you 8.0, 10.0, 12.0, 14.0


# Create histogram for charm hadrons per event for each ptHat value
histograms = []
colors = [rt.kRed, rt.kBlue, rt.kGreen+2, rt.kMagenta, rt.kOrange+7, 
          rt.kCyan+1, rt.kYellow+2, rt.kViolet+1, rt.kTeal+1, rt.kSpring+1, rt.kGray+2]


# Process each ptHat value
for pt_hat, color in zip(pt_hat_values, colors):
    filename = f"genpt{pt_hat:.1f}.root"
    print(f"\nProcessing {filename}")

    
    # Open ROOT file and get tree
    f = rt.TFile(filename)
    tree = f.Get("EventTree")
    
    # Create TClonesArray to hold particles
    particles = rt.TClonesArray("TParticle")
    tree.SetBranchAddress("Particles", particles)
    
    # Process events
    total_charm = 0
    total_events = tree.GetEntries()
    events_with_charm = 0
    
    for i in tqdm(range(total_events), desc=f"ptHat {pt_hat:.1f}"):
        tree.GetEntry(i)
        charm_in_event = 0
        charms = []
        # Count charm hadrons in this event
        for j in range(particles.GetEntries()):
            p = particles.At(j)
            if abs(p.GetPdgCode()) in final_charm_hadrons:
                charm_in_event += 1
                charms.append(p)
        if charm_in_event ==3 :
            print(f"Event {i} has {charm_in_event} charm hadrons")
            print(f"PDG codes: {[p.GetPdgCode() for p in charms]}")
        total_charm += charm_in_event
        if charm_in_event > 0:
            events_with_charm += 1
    

    
    # Print statistics for this ptHat value
    print(f"ptHat = {pt_hat:.1f}:")
    print(f"  Total events: {total_events}")
    print(f"  Events with charm: {events_with_charm} ({events_with_charm/total_events*100:.2f}%)")
    print(f"  Total charm hadrons: {total_charm}")
    print(f"  Average charm hadrons per event: {total_charm/total_events:.4f}")
    
    f.Close()




Processing genpt4.0.root


ptHat 4.0:  34%|███▍      | 345/1000 [00:00<00:00, 662.96it/s]

Event 235 has 3 charm hadrons
PDG codes: [-431, 421, -411]
Event 309 has 3 charm hadrons
PDG codes: [431, 421, -421]
Event 340 has 3 charm hadrons
PDG codes: [411, 421, -421]
Event 344 has 3 charm hadrons
PDG codes: [421, -411, -421]


ptHat 4.0:  50%|█████     | 505/1000 [00:00<00:00, 736.45it/s]

Event 400 has 3 charm hadrons
PDG codes: [421, -411, -431]


ptHat 4.0: 100%|██████████| 1000/1000 [00:01<00:00, 746.41it/s]


ptHat = 4.0:
  Total events: 1000
  Events with charm: 203 (20.30%)
  Total charm hadrons: 491
  Average charm hadrons per event: 0.4910


see the jump at 3? indicates that theres probably mass of 1.5 GeV for charm particles

In [6]:
charm_pdgs = {
    # D mesons
    411: "D+",
    -411: "D-",
    421: "D0",
    -421: "D0_bar",
    413: "D*+",
    -413: "D*-",
    423: "D*0",
    -423: "D*0_bar",
    431: "Ds+",
    -431: "Ds-",
    
    # Charmed baryons
    4122: "Λc+",
    -4122: "Λc-",
    4222: "Σc++",
    4212: "Σc+",
    4112: "Σc0",
    4224: "Σc*++",
    4214: "Σc*+",
    4114: "Σc*0"
}

pt_hat_values = np.linspace(0, 20, 11)

# Create histograms
h_total = rt.TH1F("h_total", "Total Charm vs p_{T}^{hat};p_{T}^{hat} [GeV];N_{charm}/event", 
                  len(pt_hat_values)-1, pt_hat_values[0], pt_hat_values[-1])
h_types = {}
for pdg, name in charm_pdgs.items():
    h_types[pdg] = rt.TH1F(f"h_{pdg}", f"{name} vs p_{{T}}^{{hat}};p_{{T}}^{{hat}} [GeV];Counts/event", 
                          len(pt_hat_values)-1, pt_hat_values[0], pt_hat_values[-1])

# Analyze each file
for pt_hat in tqdm(pt_hat_values, desc="Analyzing files"):
    filename = f"genpt{pt_hat:.1f}.root"
    f = rt.TFile(filename)
    tree = f.Get("EventTree")
    
    # Create TClonesArray to hold particles
    particles = rt.TClonesArray("TParticle")
    tree.SetBranchAddress("Particles", particles)
    
    # Count charm particles
    charm_counts = {pdg: 0 for pdg in charm_pdgs}
    n_events = tree.GetEntries()
    
    for i in range(n_events):
        tree.GetEntry(i)
        
        for j in range(particles.GetEntriesFast()):
            p = particles.At(j)
            pdg = p.GetPdgCode()
            
            if pdg in charm_pdgs:
                charm_counts[pdg] += 1
    
    # Fill histograms (normalize by number of events)
    total_charm = sum(charm_counts.values())
    h_total.Fill(pt_hat, total_charm/n_events)
    
    for pdg, count in charm_counts.items():
        h_types[pdg].Fill(pt_hat, count/n_events)
    
    # Print results for this pT value
    print(f"\nResults for pTHatMin = {pt_hat:.1f} GeV:")
    print("PDG Code | Particle | Count")
    print("-------------------------")
    for pdg, count in sorted(charm_counts.items(), key=lambda x: x[1], reverse=True):
        if count > 0:  # Only print if particles were found
            print(f"{pdg:<8} | {charm_pdgs[pdg]:<8} | {count}")
    print(f"Total charm particles: {total_charm}")
    print(f"Charm particles per event: {total_charm/n_events:.3f}")
    
    f.Close()

# Create canvas for total charm production
c1 = rt.TCanvas("c1", "Total Charm Production", 800, 600)
h_total.SetLineColor(rt.kBlue)
h_total.SetLineWidth(2)
h_total.Draw("HIST")
c1.SetLogy(True)
c1.Draw()

# Create canvas for individual charm hadrons
c2 = rt.TCanvas("c2", "Charm Hadron Species", 1200, 800)
legend = rt.TLegend(0.7, 0.7, 0.9, 0.9)

# Plot top 5 most common charm hadrons
colors = [rt.kRed, rt.kBlue, rt.kGreen+2, rt.kMagenta, rt.kOrange+7]
first = True

# Get total counts for each particle type to find top 5
total_counts = {}
for pdg in charm_pdgs:
    hist = h_types[pdg]
    total_counts[pdg] = sum(hist.GetBinContent(bin) for bin in range(1, hist.GetNbinsX()+1))

# Sort by total counts and get top 5
top_5_pdgs = sorted(total_counts.items(), key=lambda x: x[1], reverse=True)[:5]

for (pdg, _), color in zip(top_5_pdgs, colors):
    hist = h_types[pdg]
    hist.SetLineColor(color)
    hist.SetLineWidth(2)
    if first:
        hist.Draw("HIST")
        first = False
    else:
        hist.Draw("HIST SAME")
    legend.AddEntry(hist, charm_pdgs[pdg], "l")

legend.Draw()
c2.SetLogy(True)
c2.Draw()

Analyzing files:   9%|▉         | 1/11 [00:00<00:04,  2.01it/s]


Results for pTHatMin = 0.0 GeV:
PDG Code | Particle | Count
-------------------------
-421     | D0_bar   | 21
421      | D0       | 18
-411     | D-       | 12
-423     | D*0_bar  | 11
411      | D+       | 10
-413     | D*-      | 7
423      | D*0      | 7
431      | Ds+      | 5
413      | D*+      | 3
-431     | Ds-      | 2
4122     | Λc+      | 1
Total charm particles: 97
Charm particles per event: 0.097


Analyzing files:  18%|█▊        | 2/11 [00:01<00:05,  1.79it/s]


Results for pTHatMin = 2.0 GeV:
PDG Code | Particle | Count
-------------------------
-421     | D0_bar   | 39
421      | D0       | 34
-411     | D-       | 16
411      | D+       | 14
-423     | D*0_bar  | 14
413      | D*+      | 13
-413     | D*-      | 10
423      | D*0      | 9
431      | Ds+      | 8
4122     | Λc+      | 4
-431     | Ds-      | 3
-4122    | Λc-      | 2
4222     | Σc++     | 1
4224     | Σc*++    | 1
Total charm particles: 168
Charm particles per event: 0.168


Analyzing files:  27%|██▋       | 3/11 [00:02<00:07,  1.12it/s]

...

Analyzing files:  91%|█████████ | 10/11 [00:17<00:02,  2.18s/it]


Results for pTHatMin = 18.0 GeV:
PDG Code | Particle | Count
-------------------------
421      | D0       | 382
-421     | D0_bar   | 371
411      | D+       | 193
-411     | D-       | 187
413      | D*+      | 145
423      | D*0      | 135
-423     | D*0_bar  | 132
-413     | D*-      | 118
-431     | Ds-      | 73
431      | Ds+      | 62
-4122    | Λc-      | 34
4122     | Λc+      | 29
4224     | Σc*++    | 3
4222     | Σc++     | 2
4212     | Σc+      | 2
4114     | Σc*0     | 2
4112     | Σc0      | 1
Total charm particles: 1871
Charm particles per event: 1.871


Analyzing files: 100%|██████████| 11/11 [00:19<00:00,  1.80s/it]


Results for pTHatMin = 20.0 GeV:
PDG Code | Particle | Count
-------------------------
-421     | D0_bar   | 352
421      | D0       | 342
411      | D+       | 178
-411     | D-       | 171
-413     | D*-      | 141
423      | D*0      | 124
-423     | D*0_bar  | 124
413      | D*+      | 114
431      | Ds+      | 79
-431     | Ds-      | 79
4122     | Λc+      | 29
-4122    | Λc-      | 18
4214     | Σc*+     | 3
4222     | Σc++     | 2
4114     | Σc*0     | 2
4212     | Σc+      | 1
4224     | Σc*++    | 1
Total charm particles: 1760
Charm particles per event: 1.760



Warning in <TROOT::Append>: Replacing existing TH1: h_total (Potential memory leak).


In [14]:
pt_hat_values = np.linspace(0, 20, 11)

# Create histogram
h_total = rt.TH1F("h_total", "Total Charm vs p_{T}^{hat};p_{T}^{hat} [GeV];N_{charm}/event", 
                  len(pt_hat_values)-1, pt_hat_values[0], pt_hat_values[-1])

# Analyze each file
for pt_hat in tqdm(pt_hat_values, desc="Analyzing files"):
    filename = f"genpt{pt_hat:.1f}.root"
    f = rt.TFile(filename)
    tree = f.Get("EventTree")
    
    # Create TClonesArray to hold particles
    particles = rt.TClonesArray("TParticle")
    tree.SetBranchAddress("Particles", particles)
    
    # Count charm particles
    charm_counts = {pdg: 0 for pdg in charm_pdgs}
    n_events = tree.GetEntries()
    
    for i in range(n_events):
        tree.GetEntry(i)
        
        for j in range(particles.GetEntriesFast()):
            p = particles.At(j)
            pdg = p.GetPdgCode()
            
            if pdg in charm_pdgs:
                charm_counts[pdg] += 1
    
    # Calculate total charm and normalized value
    total_charm = sum(charm_counts.values())
    normalized_value = total_charm/n_events
    
    # Find the bin for this pt_hat value and set content and error
    bin = h_total.Fill(pt_hat, normalized_value)
    if total_charm > 0:
        error = rt.sqrt(total_charm)/n_events
        h_total.SetBinError(bin, error)
    
    # Print results for this pT value
    print(f"\nResults for pTHatMin = {pt_hat:.1f} GeV:")
    print(f"Total charm: {total_charm}")
    print(f"N events: {n_events}")
    print(f"Normalized value: {normalized_value:.6f}")
    print(f"Error: {error if total_charm > 0 else 0:.6f}")
    
    f.Close()

# Create and setup canvas
c1 = rt.TCanvas("c1", "Total Charm Production", 800, 600)
c1.SetLogy(True)

# Style the histogram
h_total.SetLineColor(rt.kBlue)
h_total.SetLineWidth(2)
h_total.SetMarkerStyle(20)  # Circular markers
h_total.SetMarkerColor(rt.kBlue)
h_total.SetMarkerSize(1.0)

# Set error bar style
rt.gStyle.SetEndErrorSize(3)  # Size of error bar end caps

# Draw with points and error bars
h_total.Draw("EP")  # E for error bars, P for points
c1.Draw()

# Print bin contents and errors for verification
print("\nBin Contents and Errors:")
print("pT_hat | Content | Error")
print("-" * 30)
for bin in range(1, h_total.GetNbinsX() + 1):
    pt_hat = h_total.GetBinCenter(bin)
    content = h_total.GetBinContent(bin)
    error = h_total.GetBinError(bin)
    print(f"{pt_hat:6.1f} | {content:7.6f} | {error:7.6f}")

Analyzing files:   9%|▉         | 1/11 [00:00<00:02,  3.94it/s]


Results for pTHatMin = 0.0 GeV:
Total charm: 97
N events: 1000
Normalized value: 0.097000
Error: 0.009849


Analyzing files:  18%|█▊        | 2/11 [00:00<00:02,  3.23it/s]


Results for pTHatMin = 2.0 GeV:
Total charm: 168
N events: 1000
Normalized value: 0.168000
Error: 0.012961


Analyzing files:  27%|██▋       | 3/11 [00:01<00:04,  1.96it/s]

...

Analyzing files:  91%|█████████ | 10/11 [00:09<00:01,  1.24s/it]


Results for pTHatMin = 18.0 GeV:
Total charm: 1871
N events: 1000
Normalized value: 1.871000
Error: 0.043255


Analyzing files: 100%|██████████| 11/11 [00:11<00:00,  1.03s/it]


Results for pTHatMin = 20.0 GeV:
Total charm: 1760
N events: 1000
Normalized value: 1.760000
Error: 0.041952

Bin Contents and Errors:
pT_hat | Content | Error
------------------------------
   1.0 | 0.097000 | 0.009849
   3.0 | 0.168000 | 0.012961
   5.0 | 0.710000 | 0.026646
   7.0 | 1.107000 | 0.033272
   9.0 | 1.434000 | 0.037868
  11.0 | 1.484000 | 0.038523
  13.0 | 1.675000 | 0.040927
  15.0 | 1.595000 | 0.039937
  17.0 | 1.742000 | 0.041737
  19.0 | 1.871000 | 0.043255



Warning in <TROOT::Append>: Replacing existing TH1: h_total (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
